initial code collected from https://huggingface.co/aashay96/indic-gpt/blob/main/indicGPT.ipynb

In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

In [2]:

# Load a pre-trained BERT tokenizer and model
model_name = "aashay96/indic-gpt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

s:\Users\asus\anaconda3\envs\llm\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\asus\.cache\huggingface\hub\models--aashay96--indic-gpt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/3.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [3]:
# Load a dataset
#dataset = load_dataset("indic_corpus.txt")
#indic_subs.txt is the entire dataset culminated into one txt file
dataset = load_dataset("text", data_files={"train": "indic_subs.txt"}, download_mode='force_redownload')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset=dataset['train'].train_test_split(test_size=0.25)

In [8]:
dataset['train'][:10]['text']

['...ஆசிரியர் அஜய் தீட்சித் ஒரு மாதத்திற்கு இன்று சஸ்பென்ட் செய்யப்பட்டார்',
 'कितने कर्मचारी हैं। किसे यहाँ से छुट्टी दे दी गई है। कौन चला गया है।',
 'পক্ষাঘাতগ্রস্ত হয়েছে। তাকে দেখার জন্য তারা তাকে দুই দিনের প্যারোল দিয়েছে।',
 'দ্যাখ ওর হাত দ্যাখ - আঙ্গুলের চেয়েও আঙটি বেশী',
 'कृपया मुझे बस अस्पताल ले चलो।',
 'बेथ, कर्ज़ पर कितनी ईमानदारी बाकी है?',
 'is the boss around here',
 'একজন পক্ষাঘাতগ্রস্থ ভাড়াটিয়া; যার কোনো জিভ নেই।',
 'रेल, नौवहन। अब विमानन पर नज़र है। नए तरीके ज़िंदगी हैं, पुराने मौत।',
 '- -']

In [9]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128, return_overflowing_tokens=True, return_length=True)

tokenized_datasets = dataset.map(tokenize_function, 
                                 batched=True,
                                   remove_columns=["text"],
                                   )

Map:   0%|          | 0/5134797 [00:00<?, ? examples/s]

Map:   0%|          | 0/1711599 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorForLanguageModeling

In [11]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [26]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_total_limit=2,
    logging_dir="./logs",
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    warmup_steps=1000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5000,
    fp16=True,
)


In [27]:


# Initialize Trainer
trainer = Trainer(
    model=model,
    tokenizer = tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    
)


In [28]:

# Start training
trainer.train()

  0%|          | 0/120471 [00:00<?, ?it/s]

{'loss': 2.7801, 'grad_norm': 2.0424113273620605, 'learning_rate': 0.00025, 'epoch': 0.01}
{'loss': 2.8238, 'grad_norm': 1.357123613357544, 'learning_rate': 0.0005, 'epoch': 0.02}
{'loss': 2.7862, 'grad_norm': 1.0081537961959839, 'learning_rate': 0.0004999783918038744, 'epoch': 0.04}
{'loss': 2.703, 'grad_norm': 0.9383986592292786, 'learning_rate': 0.0004999135709508106, 'epoch': 0.05}
{'loss': 2.6308, 'grad_norm': 0.7738011479377747, 'learning_rate': 0.0004998055486461022, 'epoch': 0.06}
{'loss': 2.5885, 'grad_norm': 0.8141405582427979, 'learning_rate': 0.0004996543435630865, 'epoch': 0.07}
{'loss': 2.5594, 'grad_norm': 0.7068043947219849, 'learning_rate': 0.0004994599818399162, 'epoch': 0.09}
{'loss': 2.5355, 'grad_norm': 0.7171629071235657, 'learning_rate': 0.0004992224970750411, 'epoch': 0.1}
{'loss': 2.5126, 'grad_norm': 0.6872197389602661, 'learning_rate': 0.0004989419303214001, 'epoch': 0.11}
{'loss': 2.4917, 'grad_norm': 0.6967065334320068, 'learning_rate': 0.000498618330079325

KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

# RESTART IF YOU GET ANY ERROR

In [29]:
trainer.save_model()